### Cellule 1 : Importation des librairies essentielles python

In [ ]:
# Import des librairies essentielles
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


### Cellule 2 : Importation des données

In [ ]:
# Définir le chemin relatif
file_path = os.path.join("..", "data", "flipkart_com-ecommerce_sample_1050.csv")

# Chargement du dataset des produits
data = pd.read_csv(file_path)
print("Dimensions du dataset :", data.shape)


### Cellule 3 : Affichage d'un aperçu des données

In [ ]:
# Afficher les premières lignes du dataset
data.head()

### Cellule 4 : Vérification des colonnes disponibles

In [ ]:
# Afficher les noms des colonnes
print("Colonnes du dataset :", data.columns.tolist())


### Cellule 5 : Préparation du traitement du texte

In [ ]:
# Import des librairies pour le traitement du texte
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Téléchargement des ressources NLTK (à exécuter une seule fois)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


### Cellule 6 : Définition des fonctions de prétraitement

In [ ]:
# Fonction de tokenisation
def tokenizer_fct(sentence):
    sentence_clean = sentence.replace('-', ' ').replace('+', ' ').replace('/', ' ').replace('#', ' ')
    word_tokens = word_tokenize(sentence_clean)
    return word_tokens

# Liste des stop words
stop_w = set(stopwords.words('english'))
punctuations = ['[', ']', ',', '.', ':', '?', '(', ')', '!', ';', '"', "'", '&', '%', '$', '@', '#', '*', '–', '—', '...']
stop_w.update(punctuations)

# Fonction de filtrage des stop words
def stop_word_filter_fct(list_words):
    filtered_w = [w for w in list_words if not w.lower() in stop_w]
    filtered_w2 = [w for w in filtered_w if len(w) > 2]
    return filtered_w2

# Fonction de mise en minuscule et suppression de certains mots
def lower_start_fct(list_words):
    lw = [w.lower() for w in list_words if not w.startswith("@") and not w.startswith("#") and not w.startswith("http")]
    return lw

# Fonction de lemmatisation
def lemma_fct(list_words):
    lemmatizer = WordNetLemmatizer()
    lem_w = [lemmatizer.lemmatize(w) for w in list_words]
    return lem_w

# Fonction de préparation du texte pour Bag-of-Words
def transform_bow_fct(desc_text):
    word_tokens = tokenizer_fct(desc_text)
    sw = stop_word_filter_fct(word_tokens)
    lw = lower_start_fct(sw)
    transf_desc_text = ' '.join(lw)
    return transf_desc_text

# Fonction de préparation du texte pour Bag-of-Words avec lemmatisation
def transform_bow_lem_fct(desc_text):
    word_tokens = tokenizer_fct(desc_text)
    sw = stop_word_filter_fct(word_tokens)
    lw = lower_start_fct(sw)
    lem_w = lemma_fct(lw)
    transf_desc_text = ' '.join(lem_w)
    return transf_desc_text

# Fonction de préparation du texte pour les modèles de Deep Learning (BERT, USE)
def transform_dl_fct(desc_text):
    word_tokens = tokenizer_fct(desc_text)
    lw = lower_start_fct(word_tokens)
    transf_desc_text = ' '.join(lw)
    return transf_desc_text


### Cellule 7 : Application du prétraitement

In [ ]:
# Remplacer les valeurs manquantes dans 'description' par une chaîne vide
data['description'] = data['description'].fillna('')

# Application des fonctions de prétraitement
data['sentence_bow'] = data['description'].apply(lambda x: transform_bow_fct(str(x)))
data['sentence_bow_lem'] = data['description'].apply(lambda x: transform_bow_lem_fct(str(x)))
data['sentence_dl'] = data['description'].apply(lambda x: transform_dl_fct(str(x)))

print("Dimensions du dataset après prétraitement :", data.shape)


### Cellule 8 : Sauvegarde des données prétraitées (optionnel)

In [ ]:
# Sauvegarde du dataset prétraité pour une utilisation future
data.to_csv('preprocessed_product_data.csv', index=False)


### Cellule 9 : Préparation des catégories

In [ ]:
# Remplacer les valeurs manquantes dans 'product_category' par 'Unknown'
data['product_category'] = data['product_category'].fillna('Unknown')

# Liste des catégories
categories = data['product_category'].unique().tolist()
print("Catégories :", categories)

# Mapping des catégories vers des numéros
category_to_num = {category: idx for idx, category in enumerate(categories)}
data['category_num'] = data['product_category'].map(category_to_num)
y_cat_num = data['category_num'].values


### Cellule 10 : Extraction des features avec Bag-of-Words et TF-IDF


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Initialisation des vectoriseurs
cvect = CountVectorizer(stop_words='english', max_df=0.95, min_df=1)
tfidf_vect = TfidfVectorizer(stop_words='english', max_df=0.95, min_df=1)

# Sélection de la colonne à utiliser
feat = 'sentence_bow_lem'

# Ajustement et transformation des données
cv_fit = cvect.fit(data[feat])
tfidf_fit = tfidf_vect.fit(data[feat])

cv_transform = cvect.transform(data[feat])
tfidf_transform = tfidf_vect.transform(data[feat])


### Cellule 11 : Sauvegarde des embeddings Bag-of-Words

In [ ]:
# Conversion en format dense (attention à la taille de la matrice)
cv_features = cv_transform.toarray()

# Vérification ou création de la colonne 'product_id'
if 'uniq_id' in data.columns:
    data.rename(columns={'uniq_id': 'product_id'}, inplace=True)
elif 'product_id' not in data.columns:
    data['product_id'] = data.index

product_ids = data['product_id'].values

# Sauvegarde des embeddings et des identifiants de produits
np.savez_compressed('countvectorizer_embeddings.npz', product_id=product_ids, embeddings=cv_features)


### Cellule 12 : Sauvegarde des embeddings TF-IDF

In [ ]:
# Conversion en format dense
tfidf_features = tfidf_transform.toarray()

# Sauvegarde des embeddings et des identifiants de produits
np.savez_compressed('tfidf_embeddings.npz', product_id=product_ids, embeddings=tfidf_features)


### Cellule 13 : Extraction des features avec Word2Vec

In [ ]:
import gensim

# Préparation des phrases pour Word2Vec
sentences = data['sentence_bow_lem'].apply(lambda x: x.split()).tolist()

# Paramètres du modèle Word2Vec
w2v_size = 300
w2v_window = 5
w2v_min_count = 1
w2v_epochs = 100

# Entraînement du modèle Word2Vec
print("Entraînement du modèle Word2Vec...")
w2v_model = gensim.models.Word2Vec(
    sentences,
    vector_size=w2v_size,
    window=w2v_window,
    min_count=w2v_min_count,
    workers=4,
    seed=42,
    epochs=w2v_epochs
)

# Fonction pour obtenir l'embedding d'un document
def document_vector(doc):
    doc = [word for word in doc if word in w2v_model.wv.key_to_index]
    return np.mean(w2v_model.wv[doc], axis=0) if len(doc) > 0 else np.zeros(w2v_size)

# Création des embeddings pour chaque description
w2v_embeddings = np.array([document_vector(doc) for doc in sentences])

# Sauvegarde des embeddings Word2Vec
np.savez_compressed('word2vec_embeddings.npz', product_id=product_ids, embeddings=w2v_embeddings)


### Cellule 14 : Extraction des features avec BERT


In [ ]:
from transformers import AutoTokenizer, TFAutoModel
import tensorflow as tf

# Paramètres
max_length = 64
batch_size = 32
model_name = 'bert-base-uncased'

# Chargement du tokenizer et du modèle BERT
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModel.from_pretrained(model_name)

# Préparation des inputs
sentences = data['sentence_dl'].tolist()

def bert_encode(sentences, tokenizer, max_length):
    input_ids = []
    attention_masks = []

    for sentence in sentences:
        encoded = tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='tf',
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    return tf.concat(input_ids, axis=0), tf.concat(attention_masks, axis=0)

input_ids, attention_masks = bert_encode(sentences, tokenizer, max_length)

# Génération des embeddings
embeddings = []
num_examples = len(sentences)

for i in range(0, num_examples, batch_size):
    batch_input_ids = input_ids[i:i+batch_size]
    batch_attention_masks = attention_masks[i:i+batch_size]
    outputs = model(batch_input_ids, attention_mask=batch_attention_masks)
    # Utilisation de la représentation du token [CLS]
    batch_embeddings = outputs.last_hidden_state[:, 0, :].numpy()
    embeddings.append(batch_embeddings)

bert_embeddings = np.vstack(embeddings)

# Sauvegarde des embeddings BERT
np.savez_compressed('bert_embeddings.npz', product_id=product_ids, embeddings=bert_embeddings)


### Cellule 15 : Extraction des features avec Universal Sentence Encoder (USE)

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf

# Chargement du modèle USE
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Génération des embeddings
sentences = data['sentence_dl'].tolist()
batch_size = 32

embeddings = []
num_examples = len(sentences)

for i in range(0, num_examples, batch_size):
    batch_sentences = sentences[i:i+batch_size]
    batch_embeddings = embed(batch_sentences).numpy()
    embeddings.append(batch_embeddings)

use_embeddings = np.vstack(embeddings)

# Sauvegarde des embeddings USE
np.savez_compressed('use_embeddings.npz', product_id=product_ids, embeddings=use_embeddings)


### Cellule 16 : Sauvegarde des identifiants de produits et des catégories

In [ ]:
# Sauvegarde des identifiants de produits et des catégories
product_info = data[['product_id', 'product_category', 'category_num']]
product_info.to_csv('product_info.csv', index=False)


### Cellule 17 : Vérification des embeddings générés (optionnel)

In [ ]:
# Vérifier les dimensions des embeddings
print("Dimensions des embeddings CountVectorizer :", cv_features.shape)
print("Dimensions des embeddings TF-IDF :", tfidf_features.shape)
print("Dimensions des embeddings Word2Vec :", w2v_embeddings.shape)
print("Dimensions des embeddings BERT :", bert_embeddings.shape)
print("Dimensions des embeddings USE :", use_embeddings.shape)
